# 1.1 Парсинг данных

Для начало мы ипортируем библеотеки для удобной работы 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import pymorphy2

Для начало мы прочитаем каждый файл и проверим его длину строк

In [2]:
gett = pd.read_json('Data/Gett.json')
len(gett)

135

In [3]:
heart = pd.read_json('Data/Здоровье.ру.json')
len(heart)

400

In [4]:
tech = pd.read_json('Data/Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта для анализа медицинских изображений.json')
len(tech)

18

In [5]:
bot = pd.read_json('Data/Промобот.json')

нам не удалось прочитать два файла Инотех и skillbox

Теперь мы соеденяем все прочитаные файлы через concat

In [6]:
dataset = pd.concat([gett, heart, tech])

In [7]:
dataset

,info,refs,refs_quant
0,NaN,[Минтранс предлагает новую версию законопроект...,135
1,NaN,"[По информации «РИА Новости», агрегатор такси ...",135
2,NaN,"[Как сообщает издание Wired, с 1 марта 2022 го...",135
3,NaN,[Китайский агрегатор такси DiDi уйдет с россий...,135
4,NaN,"[По информации издания «Известия», антимонопол...",135
...,...,...,...
13,NaN,"[Привет, Хабр! Такое понятие, как «Глубокое об...",18
14,NaN,"[Перевод поста Филиппа Ходжетта, выступавшего ...",18
15,NaN,"[Никто не понимает, как работают самые передов...",18
16,NaN,"[Предположим, вы хотите создать приложение, ко...",18


In [8]:
dataset['refs']

0     [Минтранс предлагает новую версию законопроект...
1     [По информации «РИА Новости», агрегатор такси ...
2     [Как сообщает издание Wired, с 1 марта 2022 го...
3     [Китайский агрегатор такси DiDi уйдет с россий...
4     [По информации издания «Известия», антимонопол...
                            ...                        
13    [Привет, Хабр! Такое понятие, как «Глубокое об...
14    [Перевод поста Филиппа Ходжетта, выступавшего ...
15    [Никто не понимает, как работают самые передов...
16    [Предположим, вы хотите создать приложение, ко...
17    [Добрый день, данная статья о моем opensource ...
Name: refs, Length: 553, dtype: object

Дальше мы парсим информацию из сайта 

Подключаем и проверяем соеденение с сайтом, и ищем нужные нам статьи 

In [9]:
URL = 'https://habr.com/ru/companies/avito/articles/'

In [10]:
r = requests.get(URL)

In [11]:
soup = bs(r.text, "html.parser")

In [12]:
names = soup.find_all('div', {'class' : 'tm-article-body'})

Так же мы для удобства создаём массив с компаниями чтоб парсить было удобнее 

In [13]:
company = ['avito', 'vtb', 'vk', 'alfa', 'kaspersky', 'rosatom', 'rostelecom', 'spbifmo', 'sbermarket', 'agima', 'mkb', 'tensor']

In [14]:
for href_list in names:
    name = href_list.find('a', {'class' : 'tm-title__link'})
    print(name)

Дальше мы разделяем одну колонку(refs) на несколько других(day, timе, rate, views)

In [56]:
dataset['day'] = dataset.apply(lambda x: dataset.iloc[0][1][1]['day'], axis=1 )
dataset['time'] = dataset.apply(lambda x: dataset.iloc[0][1][1]['time'], axis=1 )
dataset['rete'] = dataset.apply(lambda x: dataset.iloc[0][1][2]['rate'], axis=1 )
dataset['views'] = dataset.apply(lambda x: dataset.iloc[0][1][2]['views'], axis=1 )

можно удалить только одну колонку так как она не нужна

In [57]:
dataset

,info,refs,refs_quant,day,time,rete,views
0,NaN,[Минтранс предлагает новую версию законопроект...,135,29,10:55,7,3500
1,NaN,"[По информации «РИА Новости», агрегатор такси ...",135,29,10:55,7,3500
2,NaN,"[Как сообщает издание Wired, с 1 марта 2022 го...",135,29,10:55,7,3500
3,NaN,[Китайский агрегатор такси DiDi уйдет с россий...,135,29,10:55,7,3500
4,NaN,"[По информации издания «Известия», антимонопол...",135,29,10:55,7,3500
...,...,...,...,...,...,...,...
13,NaN,"[Привет, Хабр! Такое понятие, как «Глубокое об...",18,29,10:55,7,3500
14,NaN,"[Перевод поста Филиппа Ходжетта, выступавшего ...",18,29,10:55,7,3500
15,NaN,"[Никто не понимает, как работают самые передов...",18,29,10:55,7,3500
16,NaN,"[Предположим, вы хотите создать приложение, ко...",18,29,10:55,7,3500


Дальше мы обрабатываем текст, приводим весь текст к одному регистру, так же удаляем все спецсимволы и лишнии пробелы

In [40]:
dataset_refs = str(dataset['refs'])

In [42]:
 dataset_refs= dataset_refs.lower()

In [48]:
dataset_refs = re.sub('[^а-яА-Я]', ' ', dataset_refs)

In [52]:
dataset_refs = re.sub(' +', ' ', dataset_refs).strip()

In [60]:
pd.to_read

AttributeError: module 'pandas' has no attribute 'to_read'